In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vila Velha,BR,2021-09-18 18:39:56,-20.3297,-40.2925,80.55,61,0,19.57
1,1,Lagoa,PT,2021-09-18 18:39:56,39.0500,-27.9833,66.76,59,13,5.99
2,2,Kaili,ID,2021-09-18 18:39:57,-3.4626,120.2147,64.42,78,55,3.40
3,3,Ribeira Grande,PT,2021-09-18 18:39:57,38.5167,-28.7000,73.80,49,20,13.80
4,4,Rikitea,PF,2021-09-18 18:39:58,-23.1203,-134.9692,72.50,77,61,13.35


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [17]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [21]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Call the figure to plot the data.
fig 

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [25]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vila Velha,BR,2021-09-18 18:39:56,-20.3297,-40.2925,80.55,61,0,19.57
7,7,Albany,US,2021-09-18 18:38:45,42.6001,-73.9662,83.91,63,4,8.77
12,12,Jamestown,US,2021-09-18 18:39:59,42.0970,-79.2353,80.29,75,40,4.00
18,18,Sur,OM,2021-09-18 18:40:05,22.5667,59.5289,83.57,71,11,14.07
23,23,Lucea,JM,2021-09-18 18:40:08,18.4510,-78.1736,88.86,78,76,17.05
27,27,Butaritari,KI,2021-09-18 18:40:09,3.0707,172.7902,82.65,70,20,16.37
31,31,Georgetown,MY,2021-09-18 18:38:18,5.4112,100.3354,78.89,92,20,3.44
34,34,Bubaque,GW,2021-09-18 18:40:10,11.2833,-15.8333,81.88,77,47,7.85
36,36,Atuona,PF,2021-09-18 18:40:11,-9.8000,-139.0333,77.94,75,2,18.28
43,43,Gat,IL,2021-09-18 18:40:13,31.6100,34.7642,79.48,100,0,8.16


In [26]:
preferred_cities_df.count()

City_ID       192
City          192
Country       192
Date          192
Lat           192
Lng           192
Max Temp      192
Humidity      192
Cloudiness    192
Wind Speed    192
dtype: int64

In [27]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vila Velha,BR,80.55,-20.3297,-40.2925,
7,Albany,US,83.91,42.6001,-73.9662,
12,Jamestown,US,80.29,42.0970,-79.2353,
18,Sur,OM,83.57,22.5667,59.5289,
23,Lucea,JM,88.86,18.4510,-78.1736,
27,Butaritari,KI,82.65,3.0707,172.7902,
31,Georgetown,MY,78.89,5.4112,100.3354,
34,Bubaque,GW,81.88,11.2833,-15.8333,
36,Atuona,PF,77.94,-9.8000,-139.0333,
43,Gat,IL,79.48,31.6100,34.7642,


In [29]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [33]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

# Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

# Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [34]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vila Velha,BR,80.55,-20.3297,-40.2925,Quality Suites Vila Velha
7,Albany,US,83.91,42.6001,-73.9662,
12,Jamestown,US,80.29,42.0970,-79.2353,
18,Sur,OM,83.57,22.5667,59.5289,
23,Lucea,JM,88.86,18.4510,-78.1736,


In [35]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [41]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [42]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))